for one piece of the experiment, we have four pointclouds and a .cvs contains position, rotation and grasp_position

step3. first appear 1 (from 0), select the image and action which 5 images before the timestep.
step4. add grasp position to .csv
step5. select the corresponding images
step6. try sam2 to do the segmentation get the mask and apply the mask
step6. create the pointcloud (4 pointcloud in total)
(step7. crop the pointcloud)


In [39]:

import os
import pandas as pd
import numpy as np
import open3d as o3d
import glob
import ast

# Path to the base directory where the subfolders are located
base_path = "/media/ubb/My Passport/robot_towel_resize"

# Path for the merged tracker file
# csv_file = os.path.join(base_path, "bag","tracker_total_zed_modified.csv")
csv_files = glob.glob(os.path.join(base_path, "**/*processed_output_zed.csv"), recursive=True)
# Prepare a DataFrame to store all changes
all_changes = pd.DataFrame(columns=[
    'name', 'subname','index', 'time', 'pc_time',
    'original_pos_left', 'original_ori_left',
    'original_pos_right', 'original_ori_right', 
    'left_gripper', 'right_gripper'
])

# def calculate_average_position(data, start_index, end_index, pos_column):
#     positions = []
#     for i in range(start_index, end_index + 1):
#         pos = data.loc[i, pos_column]
#         if isinstance(pos, str):
#             pos = eval(pos)
#         positions.append(np.array([pos['x'], pos['y'], pos['z']]))
#     return np.mean(np.array(positions), axis=0) if positions else np.array([0, 0, 0])


def position_to_nparray(position):
    if isinstance(position, str):
        position = ast.literal_eval(position)
    return np.array([position['x'], position['y'], position['z']])

def quaternion_to_nparray(quaternion):
    if isinstance(quaternion, str):
        quaternion = ast.literal_eval(quaternion)
    return np.array([quaternion['w'], quaternion['x'], quaternion['y'], quaternion['z']])

# Function to calculate the extended point
# def calculate_extended_point(position, quaternion, extension_vector):
#     position_np = position_to_nparray(position)
#     quaternion_np = quaternion_to_nparray(quaternion)
#     R = o3d.geometry.get_rotation_matrix_from_quaternion(quaternion_np)
#     extended_vector = np.dot(R, extension_vector)
#     extended_point = position_np + extended_vector
#     return extended_point.tolist()

# Function to process tracker data
def process_tracker_data(csv_path):
    global all_changes
    parts = csv_path.split(os.sep)
    # print(parts)
    name = parts[-3]  # Adjust if necessary
    subname = parts[-2] 
    changes = pd.DataFrame(columns=all_changes.columns)
    try:
        # Read the CSV file
        data = pd.read_csv(csv_path)

        data['left_pos'] = data['left_pos'].apply(np.array)
        data['left_ori'] = data['left_ori'].apply(np.array)
        data['right_pos'] = data['right_pos'].apply(np.array)
        data['right_ori'] = data['right_ori'].apply(np.array)

        # Track the last gripper state to identify changes from 0 to 1
        last_gripper_left = 0
        last_gripper_right = 0

        for index, row in data.iterrows():
            changed_left = row['left_gripper'] == 1 and last_gripper_left == 0
            changed_right = row['right_gripper'] == 1 and last_gripper_right == 0

            pc_time = data.iloc[index - 10]['time'] if index >= 10 else row['time']
            if changed_left or changed_right:
       
                changes.loc[len(changes)] = [
                    name, subname, index, row['time'], pc_time, row['left_pos'], row['left_ori'],
                    row['right_pos'], row['right_ori'], 
                    int(changed_left), int(changed_right)
                ]


            last_gripper_left = row['left_gripper']
            last_gripper_right = row['right_gripper']
        
        if not changes.empty:
            changes = changes.iloc[1:].reset_index(drop=True)
            all_changes = pd.concat([all_changes, changes])

    except Exception as e:
        print(f"Failed to process {csv_path}: {str(e)}")

for csv_file in csv_files:
    process_tracker_data(csv_file)


# Save the collected data to a new CSV file
output_csv_path = os.path.join(base_path, "gripper_changes.csv")
all_changes.to_csv(output_csv_path, index=False)
# print(f"Data saved to {output_csv_path}")

#  index from 0

In [40]:

import pandas as pd
import math

def filter_close_indices(csv_path):
    # Load the CSV file
    data = pd.read_csv(csv_path)

    # Create an empty list to store indices of rows to keep
    indices_to_drop = []

    # Get the 'index' column as a list
    indices = data['index'].tolist()
    
    # Iterate over the indices list and check differences
    for i in range(1, len(indices)):
        if abs(indices[i] - indices[i - 1]) <= 18:
            # Add both indices to the drop list if they are too close
            indices_to_drop.append(i - 1)
            indices_to_drop.append(i)

    # Remove duplicates from indices_to_drop
    indices_to_drop = list(set(indices_to_drop))

    # Drop the rows that are too close to each other
    filtered_data = data.drop(indices_to_drop).reset_index(drop=True)

    # Save the filtered data back to CSV or to a new file
    filtered_data.to_csv(csv_path.replace('.csv', '_filtered.csv'), index=False)
    print(f"Filtered data saved to {csv_path.replace('.csv', '_filtered.csv')}")

# Example usage
csv_file_path = "/media/ubb/My Passport/robot_towel_resize/gripper_changes.csv"
filter_close_indices(csv_file_path)



Filtered data saved to /media/ubb/My Passport/robot_towel_resize/gripper_changes_filtered.csv


In [41]:
import os
import shutil
import pandas as pd

# Base directory for the operations
base_path = "/media/ubb/My Passport/robot_towel_resize"
output_base = os.path.join(base_path, "grasp_point")

# Read the filtered CSV file
csv_path = os.path.join(base_path, "gripper_changes_filtered.csv")
data = pd.read_csv(csv_path)

def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def copy_images(subfolder, pc_time):
    # Directories for images
    subfolders = ['bag_images', 'bag_side_images', 'bag_top_images']
    types = ['rgb', 'depth']

    for s_folder in subfolders:
        for img_type in types:
            # Source directory
            source_dir = os.path.join(base_path, subfolder, s_folder, img_type)
            # Destination directory
            dest_dir = os.path.join(output_base, subfolder, s_folder, img_type)
            ensure_dir(dest_dir)
            
            # Filename construction
            filename = f"{pc_time}.png"
            source_file = os.path.join(source_dir, filename)
            
            # Check if the file exists before attempting to copy
            if os.path.exists(source_file):
                shutil.copy(source_file, dest_dir)
                # print(f"Copied: {source_file} to {dest_dir}")
            else:
                print(f"File not found: {source_file}")

# Iterate over the dataframe
for index, row in data.iterrows():
    subfolder = row['subname']  # Ensure this column name matches your CSV structure
    pc_time = row['pc_time']
    copy_images(subfolder, pc_time)

print("Image copying completed.")


Image copying completed.


grasping point merge

In [40]:
# Define the source and target directory paths
import os
import shutil
import pandas as pd

source_dir = '/media/ubb/My Passport/robot_towel_resize'
target_dir = '/media/ubb/My Passport/grasp_point_front_depth'



# Define the subfolders to be created under the target directory
subfolders = [
    "towel_red", "towel_pink", "towel_blue", "sweater_whiteyel", "sweater_whitesnow", "sweater_ricewhite",
    "sweater_blue", "sweater_black", "sstshirt_whitenf", "sstshirt_greyad", "sstshirt_grey", "sstshirt_green",
    "sstshirt_black", "ssshirt_yellow", "ssshirt_white", "ssshirt_pink", "ssshirt_bluebl", "ssshirt_blue",
    "napkin_red45", "napkin_red30", "napkin_pink45", "napkin_pink30", "napkin_grey45", "napkin_grey30",
    "napkin_fleek45", "napkin_fleek30", "napkin_brown45", "napkin_brown30", "lstshirt_whitehor", "lstshirt_white",
    "lstshirt_purple", "lstshirt_brown", "lstshirt_black", "lsshirt_white", "lsshirt_lightblue", "lsshirt_grey",
    "lsshirt_darkblue", "lsshirt_bluedot", "pant_pink", "pant_brown", "pant_blackjean", "pant_bluejean", 
    "pant_grey", "pant_black"
]

# Create target subfolders
for folder in subfolders:
    os.makedirs(os.path.join(target_dir, folder), exist_ok=True)

# Walk through the source directory to find and copy images
for root, dirs, files in os.walk(source_dir):
    # Check that 'grasp_point' and 'rgb' are parts of the path
    if 'depth' in root and 'bag_images' in root and 'grasp_point' in root:
        parts = root.split(os.sep)
        # Determine the target subfolder by checking folder names against the subfolder list
        target_subfolder = None
        for part in parts:
            for sf in subfolders:
                if part.startswith(sf):
                    target_subfolder = sf
                    break
            if target_subfolder:
                break

        if target_subfolder:
            for file in files:
                if file.lower().endswith('.png') or file.lower().endswith('.jpg'):
                    source_path = os.path.join(root, file)
                    target_path = os.path.join(target_dir, target_subfolder, file)
                    shutil.copy(source_path, target_path)
                    print(f'Copied {source_path} to {target_path}')

print('Finished copying images.')

Copied /media/ubb/My Passport/robot_towel_resize/grasp_point/towel_blue_1/bag_images/depth/1722118955574648592.png to /media/ubb/My Passport/grasp_point_front_depth/towel_blue/1722118955574648592.png
Copied /media/ubb/My Passport/robot_towel_resize/grasp_point/towel_blue_1/bag_images/depth/1722118964986047563.png to /media/ubb/My Passport/grasp_point_front_depth/towel_blue/1722118964986047563.png
Copied /media/ubb/My Passport/robot_towel_resize/grasp_point/towel_blue_1/bag_images/depth/1722118980579738230.png to /media/ubb/My Passport/grasp_point_front_depth/towel_blue/1722118980579738230.png
Copied /media/ubb/My Passport/robot_towel_resize/grasp_point/towel_blue_1/bag_images/depth/1722118993185985798.png to /media/ubb/My Passport/grasp_point_front_depth/towel_blue/1722118993185985798.png
Copied /media/ubb/My Passport/robot_towel_resize/grasp_point/towel_blue_1/bag_images/depth/1722119002988859609.png to /media/ubb/My Passport/grasp_point_front_depth/towel_blue/1722119002988859609.png


In [42]:
import os
import shutil
from pathlib import Path

# Define the source directories and use the first one as the target directory
source_dirs = [
    '/media/ubb/My Passport/grasp_point_front_depth',
    '/media/ubb/T9/grasp_point_front_depth'
]
target_dir = source_dirs[0]  # Use the first source directory as the target

# List of subfolders that are common across all source directories
subfolders = [
    "towel_red", "towel_pink", "towel_blue", "sweater_whiteyel", "sweater_whitesnow", "sweater_ricewhite",
    "sweater_blue", "sweater_black", "sstshirt_whitenf", "sstshirt_greyad", "sstshirt_grey", "sstshirt_green",
    "sstshirt_black", "ssshirt_yellow", "ssshirt_white", "ssshirt_pink", "ssshirt_bluebl", "ssshirt_blue",
    "napkin_red45", "napkin_red30", "napkin_pink45", "napkin_pink30", "napkin_grey45", "napkin_grey30",
    "napkin_fleek45", "napkin_fleek30", "napkin_brown45", "napkin_brown30", "lstshirt_whitehor", "lstshirt_white",
    "lstshirt_purple", "lstshirt_brown", "lstshirt_black", "lsshirt_white", "lsshirt_lightblue", "lsshirt_grey",
    "lsshirt_darkblue", "lsshirt_bluedot", "pant_pink", "pant_brown", "pant_blackjean", "pant_bluejean", 
    "pant_grey", "pant_black"
]

# Ensure the target subdirectories exist
for subfolder in subfolders:
    os.makedirs(Path(target_dir) / subfolder, exist_ok=True)

# Function to copy images from source to target
def copy_images(source, target):
    for item in os.listdir(source):
        if item.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
            source_path = os.path.join(source, item)
            target_path = os.path.join(target, item)
            if not os.path.exists(target_path):  # Check if image already exists in target
                shutil.copy2(source_path, target_path)  # Use copy2 to preserve metadata

# Iterate through each source directory and subfolder
for source_dir in source_dirs[1:]:  # Skip the first directory since it's the target
    for subfolder in subfolders:
        source_subfolder_path = Path(source_dir) / subfolder
        target_subfolder_path = Path(target_dir) / subfolder
        if source_subfolder_path.exists():
            copy_images(source_subfolder_path, target_subfolder_path)

print('Images have been successfully merged into the first source directory.')

Images have been successfully merged into the first source directory.


In [47]:
import os
import shutil

# Define the root directory
root_dir = '/media/ubb/My Passport/grasp_point_front_depth'

# Iterate through each subdirectory in the root directory
for subdir in os.listdir(root_dir):
    subdir_path = os.path.join(root_dir, subdir)
    
    # Check if it's a directory
    if os.path.isdir(subdir_path):
        # Create the 'rgb' subdirectory if it doesn't exist
        rgb_subdir_path = os.path.join(subdir_path, 'depth')
        if not os.path.exists(rgb_subdir_path):
            os.makedirs(rgb_subdir_path)
        
        # Move all .jpg and .png files into the 'rgb' subdirectory
        for file in os.listdir(subdir_path):
            if file.endswith('.jpg') or file.endswith('.png'):
                source_path = os.path.join(subdir_path, file)
                destination_path = os.path.join(rgb_subdir_path, file)
                shutil.move(source_path, destination_path)
        print(f"Processed {subdir_path}")


Processed /media/ubb/My Passport/grasp_point_front_depth/ssshirt_blue
Processed /media/ubb/My Passport/grasp_point_front_depth/lsshirt_bluedot
Processed /media/ubb/My Passport/grasp_point_front_depth/lsshirt_darkblue
Processed /media/ubb/My Passport/grasp_point_front_depth/lsshirt_grey
Processed /media/ubb/My Passport/grasp_point_front_depth/lsshirt_lightblue
Processed /media/ubb/My Passport/grasp_point_front_depth/lsshirt_white
Processed /media/ubb/My Passport/grasp_point_front_depth/lstshirt_black
Processed /media/ubb/My Passport/grasp_point_front_depth/lstshirt_brown
Processed /media/ubb/My Passport/grasp_point_front_depth/lstshirt_purple
Processed /media/ubb/My Passport/grasp_point_front_depth/lstshirt_white
Processed /media/ubb/My Passport/grasp_point_front_depth/lstshirt_whitehor
Processed /media/ubb/My Passport/grasp_point_front_depth/napkin_brown30
Processed /media/ubb/My Passport/grasp_point_front_depth/napkin_brown45
Processed /media/ubb/My Passport/grasp_point_front_depth/na

In [51]:
import os
import shutil

# Define the source and target root directories
source_root = '/media/ubb/My Passport/grasp_point_top_depth'
target_root = '/media/ubb/My Passport/grasp_point_top_rgb'

# Iterate through each subdirectory in the source root directory
for subdir in os.listdir(source_root):
    source_subdir_path = os.path.join(source_root, subdir)
    target_subdir_path = os.path.join(target_root, subdir)
    
    # Path to the 'depth' subsubfolder in the source directory
    depth_folder_path = os.path.join(source_subdir_path, 'depth')
    
    # Check if the depth subsubfolder exists
    if os.path.exists(depth_folder_path) and os.path.isdir(depth_folder_path):
        # Ensure the target subdirectory exists in the rgb directory
        if not os.path.exists(target_subdir_path):
            os.makedirs(target_subdir_path)
        
        # Define the new location for the depth folder in the target directory
        target_depth_path = os.path.join(target_subdir_path, 'depth')
        
        # Move the depth folder to the target directory
        if os.path.exists(target_depth_path):
            print(f"Depth folder already exists at {target_depth_path}")
        else:
            shutil.move(depth_folder_path, target_depth_path)
            print(f"Moved depth folder to {target_depth_path}")
    else:
        print(f"No depth folder found in {source_subdir_path}")


Moved depth folder to /media/ubb/My Passport/grasp_point_top_rgb/ssshirt_blue/depth
Moved depth folder to /media/ubb/My Passport/grasp_point_top_rgb/lsshirt_bluedot/depth
Moved depth folder to /media/ubb/My Passport/grasp_point_top_rgb/lsshirt_darkblue/depth
Moved depth folder to /media/ubb/My Passport/grasp_point_top_rgb/lsshirt_grey/depth
Moved depth folder to /media/ubb/My Passport/grasp_point_top_rgb/lsshirt_lightblue/depth
Moved depth folder to /media/ubb/My Passport/grasp_point_top_rgb/lsshirt_white/depth
Moved depth folder to /media/ubb/My Passport/grasp_point_top_rgb/lstshirt_black/depth
Moved depth folder to /media/ubb/My Passport/grasp_point_top_rgb/lstshirt_brown/depth
Moved depth folder to /media/ubb/My Passport/grasp_point_top_rgb/lstshirt_purple/depth
Moved depth folder to /media/ubb/My Passport/grasp_point_top_rgb/lstshirt_white/depth
Moved depth folder to /media/ubb/My Passport/grasp_point_top_rgb/lstshirt_whitehor/depth
Moved depth folder to /media/ubb/My Passport/gras

In [56]:
import os

# Define the root directory
root_dir = '/media/ubb/My Passport/grasp_point_robot/grasp_point_top_rgb'

# Iterate through each subdirectory in the root directory
for subdir in os.listdir(root_dir):
    depth_dir_path = os.path.join(root_dir, subdir, 'depth')
    
    # Check if the depth directory exists
    if os.path.exists(depth_dir_path) and os.path.isdir(depth_dir_path):
        # Iterate through each file in the depth directory
        for file in os.listdir(depth_dir_path):
            if file.endswith('.png'):
                # Construct full file path
                file_path = os.path.join(depth_dir_path, file)
                # Define the new file path with .jpg extension
                new_file_path = os.path.join(depth_dir_path, file[:-4] + '.jpg')
                
                # Rename the file
                os.rename(file_path, new_file_path)

                print(f"Renamed {file} to {new_file_path.split('/')[-1]}")
    else:
        print(f"No depth directory found in {os.path.join(root_dir, subdir)}")


Renamed 1722185916169799242.png to 1722185916169799242.jpg
Renamed 1722185923579668542.png to 1722185923579668542.jpg
Renamed 1722185954800213245.png to 1722185954800213245.jpg
Renamed 1722185970998862491.png to 1722185970998862491.jpg
Renamed 1722185977602964293.png to 1722185977602964293.jpg
Renamed 1722185985201038319.png to 1722185985201038319.jpg
Renamed 1722185990410640391.png to 1722185990410640391.jpg
Renamed 1722186007419291813.png to 1722186007419291813.jpg
Renamed 1722186025625282721.png to 1722186025625282721.jpg
Renamed 1722186046236539430.png to 1722186046236539430.jpg
Renamed 1722189640257394668.png to 1722189640257394668.jpg
Renamed 1722189647652932969.png to 1722189647652932969.jpg
Renamed 1722189674058454847.png to 1722189674058454847.jpg
Renamed 1722189701877009363.png to 1722189701877009363.jpg
Renamed 1722189723283275189.png to 1722189723283275189.jpg
Renamed 1722189741091438815.png to 1722189741091438815.jpg
Renamed 1722189758301370091.png to 1722189758301370091.j

In [63]:
import os
from PIL import Image
import numpy as np

# Define the base directory containing all the subfolders
base_dir = "/media/ubb/My Passport/grasp_point_robot/grasp_point_front1"

# Iterate over each subfolder in the base directory
for subfolder in os.listdir(base_dir):
    subfolder_path = os.path.join(base_dir, subfolder)
    
    if os.path.isdir(subfolder_path):
        # Define subdirectories
        mask_dir = os.path.join(subfolder_path, "masks")
        rgb_dir = os.path.join(subfolder_path, "rgb")
        depth_dir = os.path.join(subfolder_path, "depth")
        rgb_mask_dir = os.path.join(subfolder_path, "rgb_mask")
        depth_mask_dir = os.path.join(subfolder_path, "depth_mask")

        # Create directories for masked images if they do not exist
        os.makedirs(rgb_mask_dir, exist_ok=True)
        os.makedirs(depth_mask_dir, exist_ok=True)

        # Get list of mask files
        mask_files = [f for f in os.listdir(mask_dir) if f.endswith('.png')]

        for mask_file in mask_files:
            mask_path = os.path.join(mask_dir, mask_file)
            rgb_path = os.path.join(rgb_dir, mask_file.replace('_mask.png', '.jpg'))
            depth_path = os.path.join(depth_dir, mask_file.replace('_mask.png', '.jpg'))

            # Load images
            mask = Image.open(mask_path).convert('L')  # Ensure mask is in grayscale
            rgb_image = Image.open(rgb_path)
            depth_image = Image.open(depth_path)

            # Convert images to arrays for masking
            mask_array = np.array(mask) / 255
            rgb_array = np.array(rgb_image)
            depth_array = np.array(depth_image)

            # Apply mask
            masked_rgb = np.dstack([(rgb_array[:, :, i] * mask_array).astype(np.uint8) for i in range(3)])
            masked_depth = (depth_array * mask_array).astype(depth_array.dtype)

            # Convert arrays back to images
            masked_rgb_image = Image.fromarray(masked_rgb)
            masked_depth_image = Image.fromarray(masked_depth)

            # Save the masked images
            masked_rgb_image.save(os.path.join(rgb_mask_dir, f'masked_{mask_file.replace("_mask.png", ".png")}'))
            masked_depth_image.save(os.path.join(depth_mask_dir, f'masked_{mask_file.replace("_mask.png", ".png")}'))

print("Finished processing all subfolders.")

Finished processing all subfolders.


create pointcloud

In [66]:
import os
import cv2
import open3d as o3d
import numpy as np

# Base directory containing all the subfolders
base_dir = "/media/ubb/My Passport/grasp_point_robot/grasp_point_front"

# Camera intrinsics (example values, replace with actual intrinsics)
intrinsics =(1070.1000, 1070.4301, 939.8100, 549.6520)
# intrinsics = (1060.4000, 1060.0100, 946.0600, 496.6630)

def process_images_and_generate_pointclouds(rgb_dir, depth_dir, point_dir, intrinsics):
    if not os.path.exists(point_dir):
        os.makedirs(point_dir)

    rgb_files = sorted([f for f in os.listdir(rgb_dir) if f.endswith('.png')])
    depth_files = sorted([f for f in os.listdir(depth_dir) if f.endswith('.png')])
    processed_files = []
    for rgb_file, depth_file in zip(rgb_files, depth_files):
        rgb_image_path = os.path.join(rgb_dir, rgb_file)
        depth_image_path = os.path.join(depth_dir, depth_file)

        # Load and resize images
        color_raw = cv2.imread(rgb_image_path)
        color_raw = cv2.cvtColor(color_raw, cv2.COLOR_BGR2RGB)
        depth_raw = cv2.imread(depth_image_path, cv2.IMREAD_UNCHANGED)

        # Resize images to desired resolution (example: 1920x1080)
        color_hd = cv2.resize(color_raw, (1920, 1080), interpolation=cv2.INTER_CUBIC)
        depth_hd = cv2.resize(depth_raw, (1920, 1080), interpolation=cv2.INTER_NEAREST)

        # Convert images for point cloud generation
        depth_hd = o3d.geometry.Image(np.asarray(depth_hd))
        color_hd = o3d.geometry.Image(np.asarray(color_hd))

        # Generate RGBD image and point cloud
        

        rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_hd, depth_hd, convert_rgb_to_intensity=False)
        fx, fy, cx, cy = intrinsics
        pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
            rgbd_image,
            o3d.camera.PinholeCameraIntrinsic(width=1920, height=1080, fx=fx, fy=fy, cx=cx, cy=cy)
        )
        pcd.transform([[-1, 0, 0, 0], [0, -1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
        if pcd.is_empty():
            print(f"No points were found in the point cloud for {rgb_file}")
        
        
        
        pointcloud_path = os.path.join(point_dir, rgb_file.replace('.png', '.pcd'))
        try:
            o3d.io.write_point_cloud(pointcloud_path, pcd)
            processed_files.append(rgb_file.replace('.png', '.pcd'))
            # print(f"Successfully saved point cloud: {pointcloud_path}")
        except Exception as e:
            print(f"Failed to save point cloud: {pointcloud_path}. Error: {str(e)}")
    return processed_files

missing_files=[]
# Process each subfolder
for subfolder in os.listdir(base_dir):
    subfolder_path = os.path.join(base_dir, subfolder)
    if os.path.isdir(subfolder_path):
        rgb_mask_dir = os.path.join(subfolder_path, "rgb_mask")
        depth_mask_dir = os.path.join(subfolder_path, "depth_mask")
        point_dir = os.path.join(subfolder_path, "pointcloud")
        processed_files = process_images_and_generate_pointclouds(rgb_mask_dir, depth_mask_dir, point_dir, intrinsics)
        if len(processed_files) < len(os.listdir(rgb_mask_dir)):
            expected_files = [f.replace('.png', '.pcd') for f in os.listdir(rgb_mask_dir) if f.endswith('.png')]
            missing_files.extend(list(set(expected_files) - set(processed_files)))
            print(f"Missing files in {subfolder}: {list(set(expected_files) - set(processed_files))}")

print("All missing files across subfolders:", missing_files)

All missing files across subfolders: []


In [70]:
import os
import open3d as o3d

# Define the base directories
base_dir_all = "/media/ubb/My Passport/grasp_point_robot/grasp_point_front"
base_dir_top = "/media/ubb/My Passport/grasp_point_robot/grasp_point_top"
# base_dir_side = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david/baxter/838433/pointcloud_side"
base_dir_merge = "/media/ubb/My Passport/grasp_point_robot/grasp_point_merge"

# Create the merge directory if it does not exist
os.makedirs(base_dir_merge, exist_ok=True)

def quaternion_to_rotation_matrix(quat):
    """Convert quaternion [w, x, y, z] to a 4x4 rotation matrix."""
    w, x, y, z = quat
    tx = 2.0 * x
    ty = 2.0 * y
    tz = 2.0 * z
    twx = tx * w
    twy = ty * w
    twz = tz * w
    txx = tx * x
    txy = ty * x
    txz = tz * x
    tyy = ty * y
    tyz = tz * y
    tzz = tz * z
 
    return np.array([[1.0 - (tyy + tzz), txy - twz, txz + twy, 0],
                     [txy + twz, 1.0 - (txx + tzz), tyz - twx, 0],
                     [txz - twy, tyz + twx, 1.0 - (txx + tyy), 0],
                     [0, 0, 0, 1]])
 
def transform_point_cloud(pcd, translation, quaternion):
    """Apply transformation defined by a translation and quaternion rotation to a point cloud."""
    # Create the transformation matrix
    rotation_matrix = quaternion_to_rotation_matrix(quaternion)
    rotation_matrix_inversed = np.linalg.inv(rotation_matrix)
    #print(rotation_matrix)
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation_matrix[:3, :3]
    transformation_matrix[:3, 3] = translation

    # transformation_matrix = [[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]] @ transformation_matrix
 
    # Transform the point cloud
    transformed_pcd = pcd.transform(transformation_matrix) # left multiplication
    #print(np.asarray(transformed_pcd.points))

    return transformed_pcd

def load_point_cloud(file_path):
    """Utility function to load a point cloud."""
    return o3d.io.read_point_cloud(file_path)

def merge_point_clouds(front_pcd, top_pcd):
    """Merge two point clouds and return the result."""
    return front_pcd + top_pcd

# Iterate over subdirectories in the 'all' directory
for subfolder in os.listdir(base_dir_all):
    subfolder_all = os.path.join(base_dir_all, subfolder)
    subfolder_top = os.path.join(base_dir_top, subfolder)
    subfolder_merge = os.path.join(base_dir_merge, subfolder)

    # Create subdirectory in merge directory
    os.makedirs(subfolder_merge, exist_ok=True)

    if os.path.isdir(subfolder_all) and os.path.isdir(subfolder_top):
        pointcloud_dir_all = os.path.join(subfolder_all, 'pointcloud')
        pointcloud_dir_top = os.path.join(subfolder_top, 'pointcloud')

        if os.path.exists(pointcloud_dir_all) and os.path.exists(pointcloud_dir_top):
            pointcloud_files_all = {f: os.path.join(pointcloud_dir_all, f) for f in os.listdir(pointcloud_dir_all) if f.endswith('.pcd')}
            pointcloud_files_top = {f: os.path.join(pointcloud_dir_top, f) for f in os.listdir(pointcloud_dir_top) if f.endswith('.pcd')}

        # Find matching point clouds and merge them
            for file_name, path_all in pointcloud_files_all.items():
                path_top = pointcloud_files_top.get(file_name)
                if path_top:
                    # Load point clouds
                    front_pcd = load_point_cloud(path_all)
                    top_pcd = load_point_cloud(path_top)
                    translation_zed = [0.139, -0.68, -1.525] # front modified leo final
                    # quaternion_zed =  [0.689, 0.128, 0.086, -0.708] # front modified leo
                    quaternion_zed =  [0.690, 0.126, 0.084, -0.708]  # front modified leo final
                    # translation_zedtop = [-0.508, -0.313, -0.84] # top modified leo final
                    # quaternion_zedtop = [0.659, 0.608, 0.332, 0.293] # top modified leo
                    # quaternion_zedtop = [0.652, 0.615, 0.329, 0.297] # top modified leo final
                    translation_zedtop_ryd = [-0.471, -0.308, -0.81] # top modified leo
                    quaternion_zedtop_ryd = [0.655,0.611, 0.305, 0.323]

                    transformed_pcd_zed = transform_point_cloud(front_pcd, translation_zed, quaternion_zed)
                    transformed_pcd_zedtop = transform_point_cloud(top_pcd, translation_zedtop_ryd, quaternion_zedtop_ryd)
                    # transformed_pcd_zedside = transform_point_cloud(top_pcd, translation_zedside, quaternion_zedside)
                    # Merge point clouds
                    merged_pcd_2 = merge_point_clouds(transformed_pcd_zed, transformed_pcd_zedtop)
                    # merged_pcd_3 = merge_point_clouds(transformed_pcd_zed, transformed_pcd_zedtop,transformed_pcd_zedside)
                    # Save merged point cloud
                    output_path = os.path.join(subfolder_merge, file_name)
                    o3d.io.write_point_cloud(output_path, merged_pcd_2)
                    # print(f"Merged and saved point cloud to {output_path}")
                else:
                    print(f"No matching point cloud found in top folder for {file_name}, skipping.")

print("Finished merging point clouds.")

No matching point cloud found in top folder for masked_1723323479181359405.pcd, skipping.
No matching point cloud found in top folder for masked_1722266373378229998.pcd, skipping.
No matching point cloud found in top folder for masked_1723397488519073363.pcd, skipping.
No matching point cloud found in top folder for masked_1723395344667151651.pcd, skipping.
No matching point cloud found in top folder for masked_1723401886257747357.pcd, skipping.
No matching point cloud found in top folder for masked_1723214447335307106.pcd, skipping.
No matching point cloud found in top folder for masked_1723404262593429266.pcd, skipping.
No matching point cloud found in top folder for masked_1723404335026786589.pcd, skipping.
No matching point cloud found in top folder for masked_1723404412648033396.pcd, skipping.
No matching point cloud found in top folder for masked_1724703441541980350.pcd, skipping.
No matching point cloud found in top folder for masked_1722280253445047912.pcd, skipping.
No matchin

In [71]:
import os
import open3d as o3d

def downsample_pointcloud(pcd_path, output_dir, voxel_size=0.0045):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Load the point cloud
    pcd = o3d.io.read_point_cloud(pcd_path)
    # print(f"Original point count: {len(pcd.points)}")

    # Downsample the point cloud using voxel downsampling
    down_pcd = pcd.voxel_down_sample(voxel_size=voxel_size)
    # print(f"Downsampled point count: {len(down_pcd.points)}")

    # Save the downsampled point cloud
    downsampled_filename = os.path.basename(pcd_path).replace('.pcd', f'_downsampled_{voxel_size}.pcd')
    downsampled_path = os.path.join(output_dir, downsampled_filename)
    o3d.io.write_point_cloud(downsampled_path, down_pcd)
    # print(f"Downsampled point cloud saved to: {downsampled_path}")

def process_subfolders(base_dir, output_base_dir, voxel_size=0.0045):
    # Iterate through each subfolder in the base directory
    for subdir in os.listdir(base_dir):
        subdir_path = os.path.join(base_dir, subdir)
        if os.path.isdir(subdir_path):
            output_dir = os.path.join(output_base_dir, subdir)
            os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists

            # Process each .pcd file in the subdirectory
            for file in os.listdir(subdir_path):
                if file.endswith('.pcd'):
                    pcd_path = os.path.join(subdir_path, file)
                    downsample_pointcloud(pcd_path, output_dir, voxel_size)

# Define base directories
base_dir = "/media/ubb/My Passport/grasp_point_robot/grasp_point_merge"
output_base_dir = "/media/ubb/My Passport/grasp_point_robot/grasp_point_merge_downsample"

# Process all subfolders
process_subfolders(base_dir, output_base_dir)

action

In [14]:
import csv
import numpy as np
import pyquaternion as pyq
import ast

def position_to_nparray(position_str):
    if isinstance(position_str, str):
        position = ast.literal_eval(position_str)
    return np.array([position['x'], position['y'], position['z']])

def quaternion_to_nparray(quaternion_str):
    if isinstance(quaternion_str, str):
        quaternion = ast.literal_eval(quaternion_str)
    return pyq.Quaternion(quaternion['w'], quaternion['x'], quaternion['y'], quaternion['z'])

def transform_position(position):
    new_x = -position[1]
    new_y = position[2] - 1.1
    new_z = -position[0]
    transformed = np.array([new_x, new_y, new_z])
    return {'x': transformed[0], 'y': transformed[1], 'z': transformed[2]}  # Keep dictionary format

def transform_quaternion(quaternion):
    y_90 = pyq.Quaternion(axis=[0, 1, 0], angle=np.pi/2)
    z_90 = pyq.Quaternion(axis=[0, 0, 1], angle=np.pi/2)
    transformed = z_90 * (y_90 * quaternion)
    return {'x': transformed.x, 'y': transformed.y, 'z': transformed.z,'w': transformed.w}  # Keep dictionary format

def process_csv(input_file, output_file):
    with open(input_file, 'r', newline='') as infile, open(output_file, 'w', newline='') as outfile:
        reader = csv.DictReader(infile)
        # base_fields = reader.fieldnames[:-2]  # Exclude the last two gripper fields
        # new_fields = ['left', 'grasp_pos_right', 'left_gripper', 'right_gripper']
        # fieldnames =  new_fields
        # fieldnames = reader.fieldnames + ['grasp_pos_left', 'grasp_pos_right']
        # writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames)
        writer.writeheader()

        for row in reader:
            try:
                # Transform positions and quaternions, and write to the new CSV
                row['left_pos'] = str(transform_position(position_to_nparray(row['left_pos'])))
                row['right_pos'] = str(transform_position(position_to_nparray(row['right_pos'])))
                
                transformed_quat_left = transform_quaternion(quaternion_to_nparray(row['left_ori']))
                transformed_quat_right = transform_quaternion(quaternion_to_nparray(row['right_ori']))
                row['left_ori'] = str(transformed_quat_left)
                row['right_ori'] = str(transformed_quat_right)
                # row['grasp_pos_left'] = row['original_pos_left']
                # row['grasp_pos_right'] = row['original_pos_right'] 
                row['left_gripper'] = row['left_gripper']
                row['right_gripper'] = row['right_gripper']
                writer.writerow(row)
            except Exception as e:
                print(f"Error processing row, skipping: {row}, Error: {e}")
                continue  # Skip this row and continue with the next

# Example usage
input_csv = '/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david_datacollect/baxter/fold_red45/processed_output_zed.csv'
output_csv = '/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david_datacollect/baxter/fold_red45/processed_output_zed_transform.csv'
process_csv(input_csv, output_csv)


In [104]:
import os
import pandas as pd

# Define the base directory containing the subfolders
base_dir = "/media/ubb/T9/useful_data"

# Initialize an empty list to hold DataFrames
dataframes = []

# Iterate through each subdirectory in the base directory
for subdir in os.listdir(base_dir):
    subdir_path = os.path.join(base_dir, subdir)
    
    # Check if it's a directory
    if os.path.isdir(subdir_path):
        # Specify the path to the CSV file
        csv_path = os.path.join(subdir_path, "gripper_changes_filtered_new.csv")
        
        # Check if the CSV file exists
        if os.path.exists(csv_path):
            # Read the CSV file and append it to the list
            df = pd.read_csv(csv_path)
            dataframes.append(df)

# Concatenate all dataframes into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Define the path for the output CSV file
output_csv_path = "/media/ubb/T9/useful_data/combined_gripper_changes_filtered_new.csv"

# Save the combined DataFrame to a CSV file
combined_df.to_csv(output_csv_path, index=False)

print(f"Combined CSV saved to: {output_csv_path}")

Combined CSV saved to: /media/ubb/T9/useful_data/combined_gripper_changes_filtered_new.csv


In [105]:
import os
import pandas as pd

# Define the base directory containing the subfolders
csv_path1 = "/media/ubb/My Passport/useful_data/combined_gripper_changes_filtered_new.csv"
csv_path2 = "/media/ubb/T9/useful_data/combined_gripper_changes_filtered_new.csv"
# csv_path3 = "/media/ubb/Seagate Backup Plus Drive/human_demonstration/combined_gripper_changes.csv"


# Initialize an empty list to hold DataFrames
dataframes = []

df1 = pd.read_csv(csv_path1)
df2 = pd.read_csv(csv_path2)
# df3 = pd.read_csv(csv_path3)

dataframes.append(df1)
dataframes.append(df2)
# dataframes.append(df3)

# Concatenate all dataframes into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Define the path for the output CSV file
output_csv_path = "/media/ubb/My Passport/useful_data/combined_all_gripper_changes.csv"

# Save the combined DataFrame to a CSV file
combined_df.to_csv(output_csv_path, index=False)

print(f"Combined CSV saved to: {output_csv_path}")


Combined CSV saved to: /media/ubb/My Passport/useful_data/combined_all_gripper_changes.csv


In [106]:
import pandas as pd

# Path to the CSV file
csv_path = "/media/ubb/My Passport/useful_data/combined_all_gripper_changes.csv"

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_path)

# # Display the number of rows before deletion for confirmation
# print("Number of rows before deletion:", len(df))

# # Delete the row where 'pc_time' is 1718819865018888256
# df = df[df['pc_time'] != 1718819865018888256]
delete_pc_times = {'1722118993185985798', '1723386017417825822', '1722197882482904847', '1723238417374978267', '1723228129923006144', '1722203580569859292', '1723238772912021623', '1723387224479748301', '1722279954043895458', '1723404335026786589', '1722280309460189093', '1722284431411630975', '1722278698836542055', '1723290543534661215', '1723379653919954845', '1722958534813610448', '1723304961746659524', '1723308287809132246', '1723386812814993065', '1723238894845599551', '1722278842606343126', '1722266373378229998', '1723387497185237222', '1723298992149667550', '1723397488519073363', '1723294502433287646', '1724886948033116092', '1722292970328815970', '1722203265911399743', '1722282496139368487', '1722289061735656692', '1722278251771856892', '1724887050585907809', '1722292830994892103', '1723290631686909122', '1724880409127217559', '1723323479181359405', '1722280253445047912', '1723295625029818916', '1722278066578085179', '1723404262593429266', '1723402200864624245', '1724880588806284907', '1723563220905423644', '1722278554987180755', '1723232929210689736', '1723404412648033396', '1723295155843466497', '1723233724124507489', '1725058439555712029', '1722278482275852747', '1722199696444761645', '1722278193833826504', '1723758322733245891', '1723298383412408545', '1725032874783122791', '1722279015755317764', '1722292178221218155', '1725033347372194605', '1722278093000434669', '1723386826718236511', '1722280374490815664', '1723298133255645295', '1723238764704491740', '1725059411329909484', '1722958105058842923', '1723755410511511462', '1723238994199758856', '1723755188709343449', '1722278630412374102', '1722119652907742975', '1723228111103466717', '1722201615865460260', '1723232662498530692', '1723231360325312384', '1723319155758665494', '1722282686202018596', '1724882876715992657', '1723386037230558254', '1723379623795195759', '1723232823654618885', '1724881570678290534', '1723232898198101223', '1723298400124622887', '1722278104793756538', '1724891707768662799', '1723308992088959143', '1723380486237245631', '1722292878816939561', '1723232985238992843', '1723386969466179246', '1722958274217122108', '1722290091404786814', '1724884797590904699', '1724885004777158555', '1723299502351773898', '1722201454734047218', '1723385230940591164', '1723234179226542550', '1725032687111223016', '1723238828128034789', '1722280314246072111', '1725032248532083104', '1722282706415755380', '1722278009074789431', '1723386631658140160', '1722279677209244731', '1723754956212655629', '1723214447335307106', '1724703441541980350', '1724881975571287893', '1723292780925931913', '1723402196469063046', '1723232742533381013', '1722281819464769904', '1725055078095525572', '1722293164326637828', '1723385770609273205', '1724880352712310924', '1723395344667151651', '1723232681315873979', '1722967123407820163', '1723299251285570257', '1724958718774604683', '1723384900969842070', '1723563857069773916', '1723401886257747357'}

# Delete the rows where 'pc_time' is in the set of delete_pc_times
df = df[~df['pc_time'].isin(delete_pc_times)]


# Display the number of rows after deletion to confirm the row is deleted
print("Number of rows after deletion:", len(df))

# Save the updated DataFrame back to CSV
df.to_csv(csv_path, index=False)

print(f"Rows with pc_times in {delete_pc_times} have been deleted.")

Number of rows after deletion: 3573
Rows with pc_times in {'1722278842606343126', '1723563220905423644', '1723386826718236511', '1722282706415755380', '1722197882482904847', '1723298383412408545', '1723238764704491740', '1722201454734047218', '1723319155758665494', '1723323479181359405', '1723238828128034789', '1723380486237245631', '1723238772912021623', '1723290631686909122', '1723232898198101223', '1723308992088959143', '1722292178221218155', '1723384900969842070', '1723386017417825822', '1722278698836542055', '1722278009074789431', '1723404412648033396', '1723404262593429266', '1722293164326637828', '1722278251771856892', '1723563857069773916', '1722958105058842923', '1722280314246072111', '1723290543534661215', '1723233724124507489', '1724703441541980350', '1722278630412374102', '1723295625029818916', '1723299502351773898', '1723238894845599551', '1722282686202018596', '1722292970328815970', '1723295155843466497', '1722280309460189093', '1723232681315873979', '1723755188709343449'